# Lab 8 - Barcelona Bike Sharing Station Criticality Analysis

Analysis of historical data from Barcelona bike sharing stations to identify critical timeslots.

## Objectives:
- Remove invalid data (used_slots=0 AND free_slots=0)
- Calculate criticality for each (station, timeslot) pair
- Filter by minimum criticality threshold
- Join with station coordinates
- Save results sorted by criticality (desc), station (asc), weekday (asc), hour (asc)

In [ ]:
# Configuration parameters
registerPath = "sampleData 2/registerSample.csv"
stationsPath = "sampleData 2/stations.csv"
outputPath = "output_lab8/"
minCriticalityThreshold = 0.4

In [ ]:
# Read register data
registerDF = spark.read \
    .option("header", "true") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv(registerPath)

In [ ]:
# Read stations data
stationsDF = spark.read \
    .option("header", "true") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv(stationsPath)

In [ ]:
# Filter invalid data and add derived columns
cleanDF = registerDF.filter(
    ~((col("used_slots") == 0) & (col("free_slots") == 0))
).withColumn(
    "weekday", date_format(col("timestamp"), "EE")
).withColumn(
    "hour", hour(col("timestamp"))
).withColumn(
    "is_critical", when(col("free_slots") == 0, 1).otherwise(0)
)

In [ ]:
# Calculate criticality for each station-timeslot pair
criticalityDF = cleanDF.groupBy("station", "weekday", "hour").agg(
    sum("is_critical").alias("critical_count"),
    count("*").alias("total_count")
).withColumn(
    "criticality", col("critical_count") / col("total_count")
)

In [ ]:
# Filter by threshold and join with station coordinates
resultDF = criticalityDF.filter(
    col("criticality") >= minCriticalityThreshold
).join(
    stationsDF.select("id", "longitude", "latitude"),
    criticalityDF.station == stationsDF.id,
    "inner"
).select(
    "station",
    "weekday", 
    "hour",
    "criticality",
    "longitude",
    "latitude"
).orderBy(
    col("criticality").desc(),
    col("station").asc(),
    col("weekday").asc(),
    col("hour").asc()
)

In [ ]:
# Save results
resultDF.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(outputPath)

print(f"Results saved to: {outputPath}")
print(f"Number of records: {resultDF.count()}")

In [ ]:
# Display results
resultDF.show()